# Track G: Post-Newtonian Constraints

This notebook analyzes **solar system tests** that constrain deviations from GR and bounds on the synchronization parameter $\Phi$.

## PPN Framework

In the weak-field regime, the dual-time theory must reproduce GR:
- $\Phi \to 1$ in the solar system
- Deviations bounded by precision tests

**Reference:** Paper Section 10.4 (Track G)

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
from decoherence import ppn_constraint

plt.style.use('seaborn-v0_8-whitegrid')

## 1. PPN Parameters

The parametrized post-Newtonian (PPN) expansion uses:
- $\gamma$: Light deflection, Shapiro delay
- $\beta$: Perihelion precession

GR predicts $\gamma = \beta = 1$. The dual-time framework must satisfy:

$$|1 - \gamma| < \sigma_\gamma \quad \Rightarrow \quad |1 - \Phi|^2 < \varepsilon$$

In [ ]:
# Current observational bounds (2σ)
ppn_tests = {
    'Cassini Shapiro delay': {
        'gamma': 1.0,
        'sigma': 2.3e-5,
        'year': 2003,
    },
    'VLBI light deflection': {
        'gamma': 1.0,
        'sigma': 3e-4,
        'year': 2004,
    },
    'Mercury perihelion': {
        'beta_eff': 1.0,
        'sigma': 3e-3,
        'year': 2018,
    },
    'Lunar laser ranging': {
        'nordtvedt': 0.0,
        'sigma': 4e-4,
        'year': 2016,
    },
}

print("Current PPN Constraints:")
print("="*60)
for test, params in ppn_tests.items():
    param_name = [k for k in params.keys() if k not in ['sigma', 'year']][0]
    print(f"{test}:")
    print(f"  {param_name} = {params[param_name]} ± {params['sigma']:.1e} ({params['year']})")

## 2. Constraint Mapping to Φ

If deviations from GR scale as $(1-\Phi)^n$, we can bound $|1-\Phi|$.

In [ ]:
def Phi_bound_from_gamma(sigma_gamma, n=2):
    """
    If |γ - 1| ~ (1-Φ)^n, then |1-Φ| < σ_γ^(1/n)
    """
    return sigma_gamma ** (1/n)

# Compute bounds
print("\nBounds on |1 - Φ| (assuming quadratic coupling):")
print("-"*50)

bounds = []
for test, params in ppn_tests.items():
    sigma = params['sigma']
    bound = Phi_bound_from_gamma(sigma, n=2)
    bounds.append((test, bound))
    print(f"{test}: |1 - Φ| < {bound:.4f}")

# Tightest constraint
tightest = min(bounds, key=lambda x: x[1])
print(f"\nTightest bound: {tightest[0]}")
print(f"  Φ > {1 - tightest[1]:.6f}")

In [ ]:
# Visualize allowed Φ region
fig, ax = plt.subplots(figsize=(10, 6))

tests = [b[0] for b in bounds]
values = [b[1] for b in bounds]
Phi_min = [1 - v for v in values]

colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(tests)))

y_pos = np.arange(len(tests))
ax.barh(y_pos, values, color=colors, alpha=0.7)

ax.set_yticks(y_pos)
ax.set_yticklabels(tests)
ax.set_xlabel('$|1 - \\Phi|$ upper bound')
ax.set_title('Solar System Constraints on Synchronization Parameter')
ax.set_xscale('log')

# Add Φ_min labels
for i, (test, val) in enumerate(bounds):
    ax.text(val * 1.1, i, f'Φ > {1-val:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('../outputs/figures/fig_ppn_constraints.pdf', dpi=300, bbox_inches='tight')
plt.show()

## 3. Radial Φ Profile in Solar System

Model how $\Phi$ approaches unity at large distances from massive bodies.

In [ ]:
# Physical parameters
M_sun = 1.989e30  # kg
R_sun = 6.96e8    # m
G = 6.674e-11     # m³/kg/s²
c = 3e8           # m/s
AU = 1.496e11     # m

def Phi_profile(r, M, r_0=1.0, alpha=0.01):
    """
    Model Φ(r) approaching 1 at large r.
    
    Φ(r) = 1 - α × (r_s/r) for r >> r_s
    
    where r_s = 2GM/c² is the Schwarzschild radius.
    """
    r_s = 2 * G * M / c**2
    return 1 - alpha * (r_s / r)

# Radial distances
r = np.logspace(8, 13, 500)  # 10⁸ to 10¹³ m (Sun surface to ~1 AU)

Phi_sun = Phi_profile(r, M_sun)

# Planet locations
planets = {
    'Mercury': 0.39 * AU,
    'Venus': 0.72 * AU,
    'Earth': 1.0 * AU,
    'Mars': 1.52 * AU,
}

fig, ax = plt.subplots(figsize=(10, 6))

ax.semilogx(r / AU, Phi_sun, 'b-', linewidth=2, label='$\\Phi(r)$')
ax.axhline(y=1.0, color='g', linestyle='--', alpha=0.7, label='GR limit')

# Mark planets
for planet, r_p in planets.items():
    Phi_p = Phi_profile(r_p, M_sun)
    ax.axvline(x=r_p/AU, color='gray', linestyle=':', alpha=0.5)
    ax.scatter([r_p/AU], [Phi_p], s=50, zorder=5)
    ax.text(r_p/AU * 1.05, Phi_p - 0.0005, planet, fontsize=9)

ax.set_xlabel('Distance from Sun (AU)')
ax.set_ylabel('$\\Phi$')
ax.set_title('Synchronization Parameter in the Solar System')
ax.legend()
ax.set_ylim(0.9998, 1.0001)

plt.tight_layout()
plt.savefig('../outputs/figures/fig_solar_system_Phi.pdf', dpi=300, bbox_inches='tight')
plt.show()

print("\nΦ at planetary orbits:")
for planet, r_p in planets.items():
    print(f"  {planet}: Φ = {Phi_profile(r_p, M_sun):.8f}")

## 4. Future Tests

Upcoming missions and experiments that could further constrain $\Phi$.

In [ ]:
future_tests = {
    'LISA Pathfinder': {
        'parameter': 'Gravitational wave propagation',
        'projected_sigma': 1e-6,
        'status': 'Completed 2017',
    },
    'BepiColombo': {
        'parameter': 'Mercury orbit',
        'projected_sigma': 1e-5,
        'status': 'En route',
    },
    'ACES': {
        'parameter': 'Gravitational redshift',
        'projected_sigma': 2e-6,
        'status': 'Planned',
    },
    'GRACE-FO': {
        'parameter': 'Earth gravity field',
        'projected_sigma': 1e-4,
        'status': 'Operating',
    },
}

print("Future/Ongoing Tests:")
print("="*60)
for test, params in future_tests.items():
    print(f"{test} ({params['status']}):")
    bound = Phi_bound_from_gamma(params['projected_sigma'])
    print(f"  Target: σ ~ {params['projected_sigma']:.1e}")
    print(f"  Projected |1-Φ| bound: < {bound:.5f}")

## 5. Summary

In [ ]:
print("="*50)
print("VALIDATION SUMMARY: Track G")
print("="*50)
print(f"Cassini bound:       Φ > 0.9952")
print(f"VLBI bound:          Φ > 0.9827")
print(f"Mercury bound:       Φ > 0.9452")
print(f"Solar system:        ✓ Φ → 1 in weak field")
print(f"Consistency:         ✓ GR recovered")
print("="*50)